
## 📈 STOCK MOMENTUM PREDICTION SYSTEM

A Comprehensive ML-Based Approach for Identifying High-Momentum Stocks

## ⚙️ SYSTEM OVERVIEW
This system combines:
•⁠  ⁠Technical Analysis (15+ indicators)
•⁠  ⁠Machine Learning (XGBoost model)
•⁠  ⁠Backtesting Framework

## 🎯 KEY FEATURES
•⁠  ⁠Multi-timeframe momentum analysis
•⁠  ⁠Ensemble learning with XGBoost
•⁠  ⁠Automated CSV generation for previous days strategy testing

## Importing Required Libraries

In [12]:
import pandas as pd
import numpy as np
import yfinance as yf
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from typing import Dict, List, Optional
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb


## Step 1: Load Local Data
## Reads all CSVs from the local folder (e.g. 'combined_stock_data')
## Cleans and precomputes all technical indicators for each ticker
## Returns a dictionary: {symbol: DataFrame with features}

In [ ]:
def load_universe_data_from_folder(data_folder_path: str):
    """
    Loads and processes all 15-minute intraday data from a local folder of CSV files
    with filenames like 'TICKER_NS.csv'.
    """
    data_folder = Path(data_folder_path)
    if not data_folder.exists():
        print(f"[ERROR] Data folder not found at: {data_folder}")
        return {}

    print(f"Loading local 15-minute data from: {data_folder} 唐")

    all_data = {}
    # --- CHANGE: Find all relevant CSV files in the folder ---
    for file_path in data_folder.glob('*_NS.csv'):
        try:
            # --- CHANGE: Derive the ticker symbol from the filename ---
            # e.g., 'RELIANCE_NS.csv' -> 'RELIANCE.NS'
            t = file_path.stem.replace('_NS', '.NS')

            df = pd.read_csv(
                file_path,
                index_col='date',
                parse_dates=True
            )

            df.columns = [str(c).lower() for c in df.columns]
            df.index.name = "datetime"

            if not df.empty:
                all_data[t] = EnhancedIndicators.prepare_stock_for_scoring(df)
        
        except Exception as e:
            print(f"[ERROR] Failed to load or process data for {file_path.name}: {e}")

    print(f"\nSuccessfully loaded and processed data for {len(all_data)} tickers.\n")
    return all_data

## Indicators

In [ ]:
# --- Enhanced Indicators Class ---
class EnhancedIndicators:
    @staticmethod
    def _ema(values: np.ndarray, period: int) -> np.ndarray:
        n = len(values)
        ema = np.full(n, np.nan, dtype=float)
        if n == 0 or period <= 0:
            return ema
        if n >= period:
            seed_idx = period - 1
            seed = np.mean(values[:period])
        else:
            seed_idx = 0
            seed = np.mean(values[: max(1, n)])
        ema[seed_idx] = seed
        alpha = 2.0 / (period + 1.0)
        prev = seed
        for i in range(seed_idx + 1, n):
            x = values[i]
            prev = (x - prev) * alpha + prev
            ema[i] = prev
        return ema

    @staticmethod
    def add_garman_klass_volatility(df, period: int = 20):
        """
        Add Garman-Klass Volatility estimator
        """
        open_prices = df['open'].to_numpy(dtype=float)
        high = df['high'].to_numpy(dtype=float)
        low = df['low'].to_numpy(dtype=float)
        close = df['close'].to_numpy(dtype=float)

        n = len(df)
        gk_volatility = np.full(n, np.nan, dtype=float)

        if n < period:
            df['gk_volatility'] = gk_volatility
            return df

        # Calculate the Garman-Klass components for each period
        for i in range(period, n):
            start_idx = i - period + 1
            end_idx = i + 1

            # Extract the window of data
            window_open = open_prices[start_idx:end_idx]
            window_high = high[start_idx:end_idx]
            window_low = low[start_idx:end_idx]
            window_close = close[start_idx:end_idx]

            # Calculate the components
            log_hl_ratio = np.log(window_high / window_low)
            log_co_ratio = np.log(window_close / window_open)

            # Garman-Klass formula
            gk_components = 0.5 * (log_hl_ratio ** 2) - (2 * np.log(2) - 1) * (log_co_ratio ** 2)

            # Sum and annualize (assuming daily data - 252 trading days)
            gk_variance = np.sum(gk_components) / period
            gk_volatility[i] = np.sqrt(gk_variance * 252)

        df['gk_volatility'] = gk_volatility
        return df

    @staticmethod
    def add_adx(df, period: int = 14):
        high = df['high'].to_numpy(dtype=float)
        low = df['low'].to_numpy(dtype=float)
        close = df['close'].to_numpy(dtype=float)

        n = len(df)
        adx = np.full(n, np.nan, dtype=float)
        pdi = np.full(n, np.nan, dtype=float)
        mdi = np.full(n, np.nan, dtype=float)

        if n < period + 1:
            df['adx'] = adx
            df['+di'] = pdi
            df['-di'] = mdi
            return df

        # Calculate True Range, +DM, -DM
        tr = np.zeros(n)
        plus_dm = np.zeros(n)
        minus_dm = np.zeros(n)

        for i in range(1, n):
            # True Range
            tr1 = high[i] - low[i]
            tr2 = abs(high[i] - close[i-1])
            tr3 = abs(low[i] - close[i-1])
            tr[i] = max(tr1, tr2, tr3)

            # Directional Movement
            up_move = high[i] - high[i-1]
            down_move = low[i-1] - low[i]

            if up_move > down_move and up_move > 0:
                plus_dm[i] = up_move
            else:
                plus_dm[i] = 0

            if down_move > up_move and down_move > 0:
                minus_dm[i] = down_move
            else:
                minus_dm[i] = 0

        # Smooth the values
        tr_smooth = np.zeros(n)
        plus_dm_smooth = np.zeros(n)
        minus_dm_smooth = np.zeros(n)

        tr_smooth[period] = np.sum(tr[1:period+1])
        plus_dm_smooth[period] = np.sum(plus_dm[1:period+1])
        minus_dm_smooth[period] = np.sum(minus_dm[1:period+1])

        for i in range(period + 1, n):
            tr_smooth[i] = tr_smooth[i-1] - (tr_smooth[i-1] / period) + tr[i]
            plus_dm_smooth[i] = plus_dm_smooth[i-1] - (plus_dm_smooth[i-1] / period) + plus_dm[i]
            minus_dm_smooth[i] = minus_dm_smooth[i-1] - (minus_dm_smooth[i-1] / period) + minus_dm[i]

        # Calculate +DI and -DI
        for i in range(period, n):
            if tr_smooth[i] != 0:
                pdi[i] = 100 * (plus_dm_smooth[i] / tr_smooth[i])
                mdi[i] = 100 * (minus_dm_smooth[i] / tr_smooth[i])

        # Calculate DX and ADX
        dx = np.zeros(n)
        for i in range(period, n):
            if pdi[i] + mdi[i] != 0:
                dx[i] = 100 * abs(pdi[i] - mdi[i]) / (pdi[i] + mdi[i])

        # Calculate ADX
        adx[period] = np.mean(dx[period:period+period])
        for i in range(period + 1, n):
            adx[i] = (adx[i-1] * (period - 1) + dx[i]) / period

        df['adx'] = adx
        df['+di'] = pdi
        df['-di'] = mdi

        return df

    @staticmethod
    def add_intraday_return(df):
        if 'open' in df.columns and 'close' in df.columns:
            df['intraday_return'] = (df['close'] - df['open']) / df['open']
        return df

    @staticmethod
    def add_ema_slope(df, period: int = 20):
        if 'close' in df.columns:
            df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
            df['ema_20_slope'] = (df['ema_20'] - df['ema_20'].shift(5)) / 5
        return df

    @staticmethod
    def prepare_stock_for_scoring(df: pd.DataFrame) -> pd.DataFrame:
        """Calculate all required indicators for z-score calculation AND momentum prediction"""
        # Use Garman-Klass volatility instead of ATR
        df = EnhancedIndicators.add_garman_klass_volatility(df)
        df = EnhancedIndicators.add_adx(df)
        df = EnhancedIndicators.add_intraday_return(df)
        df = EnhancedIndicators.add_ema_slope(df)

        # NEW: Add momentum features for momentum prediction
        df = MomentumFeatureEngineer.add_momentum_features(df)
        df = MomentumFeatureEngineer.add_advanced_oscillators(df)
        df = add_choppiness_metric(df)

        df = MomentumFeatureEngineer.add_momentum_targets(df) 

        # Calculate additional parameters for z-scores
        if 'volume' in df.columns:
            df['volume_10d_mean'] = df['volume'].rolling(window=10).mean()
            df['volume_10d_std'] = df['volume'].rolling(window=10).std()
            volume_std_safe = df['volume_10d_std'].copy()
            volume_std_safe[volume_std_safe == 0] = 1
            df['volume_z_ready'] = (df['volume'] - df['volume_10d_mean']) / volume_std_safe

        if 'adx' in df.columns:
            df['adx_normalized'] = (df['adx'] - 25) / 25
            df['adx_normalized'] = np.clip(df['adx_normalized'], -1, 1)

        return df

## Momentum Feature Engineer


In [ ]:

class MomentumFeatureEngineer:
    @staticmethod
    def add_momentum_features(df):
        """Add features specifically for momentum/trend prediction"""

        # Price-based momentum features
        df['momentum_1'] = df['close'].pct_change(1)
        df['momentum_3'] = df['close'].pct_change(3)
        df['momentum_5'] = df['close'].pct_change(5)
        df['momentum_10'] = df['close'].pct_change(10)

        # Volume momentum
        df['volume_momentum'] = df['volume'].pct_change(5)
        df['volume_ratio'] = df['volume'] / df['volume'].rolling(20).mean()

        # Volatility features
        df['volatility_5'] = df['close'].pct_change().rolling(5).std()
        df['volatility_20'] = df['close'].pct_change().rolling(20).std()
        df['volatility_ratio'] = df['volatility_5'] / df['volatility_20']

        # Range-based features (important for trending)
        df['normalized_range'] = (df['high'] - df['low']) / df['close']
        df['range_ratio'] = (df['high'] - df['low']) / (df['high'].rolling(5).max() - df['low'].rolling(5).min())

        # Trend strength features
        df['trend_strength'] = (df['close'] - df['close'].rolling(20).mean()) / df['close'].rolling(20).std()
        df['price_vs_ema'] = df['close'] / df['close'].ewm(span=20).mean() - 1

        # Support/Resistance features
        df['distance_from_high'] = (df['high'].rolling(20).max() - df['close']) / df['close']
        df['distance_from_low'] = (df['close'] - df['low'].rolling(20).min()) / df['close']

        return df

    @staticmethod
    def add_advanced_oscillators(df):
        """Add momentum oscillators"""

        # RSI
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['rsi'] = 100 - (100 / (1 + rs))

        # Stochastic
        low_14 = df['low'].rolling(14).min()
        high_14 = df['high'].rolling(14).max()
        df['stoch'] = 100 * (df['close'] - low_14) / (high_14 - low_14)

        # MACD
        ema_12 = df['close'].ewm(span=12).mean()
        ema_26 = df['close'].ewm(span=26).mean()
        df['macd'] = ema_12 - ema_26
        df['macd_signal'] = df['macd'].ewm(span=9).mean()
        df['macd_histogram'] = df['macd'] - df['macd_signal']

        # Rate of Change
        df['roc'] = (df['close'] / df['close'].shift(10) - 1) * 100

        return df

    @staticmethod
    def add_momentum_targets(df, lookforward_periods=25):
        """
        Add targets that capture momentum and trending behavior
        """
        # 1. Trend Strength - Absolute price movement magnitude
        df['future_trend_strength'] = (df['close'].shift(-lookforward_periods) - df['close']).abs() / df['close']

        # 2. Directional Consistency - How consistently the stock moves in one direction
        future_returns = df['close'].pct_change(lookforward_periods).shift(-lookforward_periods)
        df['future_directional_consistency'] = future_returns.rolling(3, min_periods=1).std()  # Lower = more trending

        # 3. Momentum Persistence - Whether current momentum continues
        current_momentum = df['close'].pct_change(3)
        future_momentum = df['close'].pct_change(3).shift(-lookforward_periods)
        # Use rolling correlation for better persistence measure
        df['future_momentum_persistence'] = (current_momentum * future_momentum > 0).astype(float)

        # 4. Volatility Regime - High volatility often indicates trends
        df['future_volatility'] = df['close'].pct_change().rolling(lookforward_periods, min_periods=1).std().shift(-lookforward_periods)

        # Replace any inf values
        df = df.replace([np.inf, -np.inf], np.nan)

        return df


def add_choppiness_metric(df, period=14):
    """
    Choppiness Index - Lower values indicate stronger trends
    """
    high = df['high']
    low = df['low']
    close = df['close']

    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    true_range = np.maximum(tr1, np.maximum(tr2, tr3))

    atr = true_range.rolling(period).mean()
    highest_high = high.rolling(period).max()
    lowest_low = low.rolling(period).min()

    df['choppiness'] = 100 * np.log10(atr.rolling(period).sum() / (highest_high - lowest_low)) / np.log10(period)
    return df

## Momentum Predictor

In [ ]:

class MomentumPredictor:
    def __init__(self):
        self.models = {}
        self.feature_columns = []

    def clean_data(self, df, feature_cols, target_cols):
        """Clean data by removing inf, NaN, and extreme values"""
        # Create a copy of the relevant columns
        data = df[feature_cols + target_cols].copy()

        # Replace inf with NaN
        data = data.replace([np.inf, -np.inf], np.nan)

        # Remove rows with any NaN values
        data = data.dropna()

        # Remove extreme outliers (beyond 5 standard deviations)
        for col in feature_cols:
            if col in data.columns:
                mean = data[col].mean()
                std = data[col].std()
                if std > 0:  # Avoid division by zero
                    z_scores = np.abs((data[col] - mean) / std)
                    data = data[z_scores < 5]

        return data

    def prepare_features_targets(self, df):
        """
        Defines feature and target columns from a pre-processed DataFrame.
        No calculations are performed here.
        """
        # Define feature columns
        feature_cols = [
            'momentum_1', 'momentum_3', 'momentum_5', 'momentum_10',
            'volume_momentum', 'volume_ratio',
            'volatility_5', 'volatility_20', 'volatility_ratio',
            'normalized_range', 'range_ratio',
            'trend_strength', 'price_vs_ema',
            'distance_from_high', 'distance_from_low',
            'rsi', 'stoch', 'macd', 'macd_histogram', 'roc',
            'choppiness', 'adx', '+di', '-di', 'gk_volatility'
        ]

        # Target columns
        target_cols = [
            'future_trend_strength',
            'future_directional_consistency',
            'future_momentum_persistence',
            'future_volatility'
        ]

        # Just return the DataFrame and the column lists
        return df, feature_cols, target_cols

    def train_models(self, universe_data):
        """Train models for each stock"""

        for symbol, df in universe_data.items():
            try:
                # Prepare data
                df_processed, feature_cols, target_cols = self.prepare_features_targets(df.copy())
                self.feature_columns = feature_cols

                # Clean the data
                valid_data = self.clean_data(df_processed, feature_cols, target_cols)

                if len(valid_data) < 50:  # Minimum data points
                    print(f"Skipping {symbol}: insufficient data after cleaning ({len(valid_data)} samples)")
                    continue

                X = valid_data[feature_cols]
                y = valid_data[target_cols]

                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.2, shuffle=False
                )

                # Train XGBoost model with error handling
                model = xgb.XGBRegressor(
                    n_estimators=40,  # Reduced for faster training
                    max_depth=3,      # Reduced to prevent overfitting
                    learning_rate=0.1,
                    objective='reg:squarederror',
                    random_state=42
                )

                # Use MultiOutputRegressor for multiple targets
                multi_model = MultiOutputRegressor(model)
                multi_model.fit(X_train, y_train)

                self.models[symbol] = multi_model

                print(f"Trained momentum model for {symbol} - {len(valid_data)} samples")

            except Exception as e:
                print(f"Error training momentum model for {symbol}: {str(e)[:100]}")  # Truncate long errors

    def predict_momentum_metrics(self, symbol, current_features):
        """Predict momentum metrics for current market state"""
        if symbol not in self.models:
            return None

        try:
            # Ensure we have the right features and handle missing columns
            available_features = [col for col in self.feature_columns if col in current_features.columns]
            if len(available_features) != len(self.feature_columns):
                missing = set(self.feature_columns) - set(available_features)
                print(f"Missing features for {symbol}: {missing}")
                return None

            # Clean current features
            current_data = current_features[self.feature_columns].copy()
            current_data = current_data.replace([np.inf, -np.inf], np.nan)

            # Check for any NaN values in current features
            if current_data.isna().any().any():
                print(f"NaN values in current features for {symbol}")
                return None

            prediction = self.models[symbol].predict(current_data)[0]

            return {
                'trend_strength': float(prediction[0]),
                'directional_consistency': float(prediction[1]),
                'momentum_persistence': float(prediction[2]),
                'volatility': float(prediction[3])
            }
        except Exception as e:
            print(f"Error predicting for {symbol}: {e}")
            return None

## Main Execution to generate momentum ranking files for specified date interval

In [ ]:
# --- Main Execution ---
# --- Backtest CSV Generator ---
def generate_backtest_csvs():
    """
    Generates daily 15-minute momentum rankings with a rolling training window.
    """
    # 1. Define the path to your data folder
    YOUR_DATA_FOLDER = "combined_stock_data"  # <--- SET YOUR FOLDER NAME HERE

    # 2. Call the new function to load data from the folder
    all_universe_data = load_universe_data_from_folder(YOUR_DATA_FOLDER)
    
    if not all_universe_data:
        print("No local data loaded. Exiting.")
        return

    # Create output folder
    backtest_dir = Path("backtest_data_15min")
    backtest_dir.mkdir(parents=True, exist_ok=True)
    
    # --- Logic to determine when to train and predict ---
    # We will train once per day using the data available at market open (e.g., 9:15 AM)
    
    # Get all unique dates from a sample ticker
    sample_symbol = next(iter(all_universe_data))
    all_timestamps = all_universe_data[sample_symbol].index
    unique_dates = pd.to_datetime(all_timestamps.normalize().unique())

    # --- FILTER FOR CUSTOM DATE RANGE ---
    start_date = pd.to_datetime("2024-09-01")   # September 1, 2024
    end_date   = pd.to_datetime("2025-09-30")   # September 30, 2025

    # Keep only dates in this range
    unique_dates = unique_dates[(unique_dates >= start_date) & (unique_dates <= end_date)]


    print(f"Found {len(unique_dates)} unique trading days in the data. Starting backtest...\n")

    # --- Main Backtest Loop ---
    for i, prediction_date in enumerate(unique_dates):
        prediction_date_str = prediction_date.strftime("%Y-%m-%d")
        print(f"Processing {i+1}/{len(unique_dates)}: {prediction_date_str}")
        
        try:
            # The training data is everything UP TO this day's open
            # We use tail(250) to get the last ~10 days of 15-min bars
            training_slice = {}
            for symbol, df in all_universe_data.items():
                # Get all data strictly before the current prediction day
                end_of_training = df.index < prediction_date
                if end_of_training.any():
                    # Take the last 250 bars for training
                    training_data = df.loc[end_of_training].tail(250)
                    if not training_data.empty and len(training_data) > 50: # Ensure enough data
                        training_slice[symbol] = training_data

            if not training_slice:
                print(f"  [SKIP] Not enough training data before {prediction_date_str}.")
                continue
            
            # Train the momentum model on the sliced data (~250 bars)
            momentum_predictor = MomentumPredictor()
            momentum_predictor.train_models(training_slice)

            # Predict using the very last bar of the training data
            momentum_predictions = {}
            for symbol, df in training_slice.items():
                last_bar = df.iloc[[-1]] # Get the last row as a DataFrame
                prediction = momentum_predictor.predict_momentum_metrics(symbol, last_bar)
                if prediction:
                    momentum_predictions[symbol] = prediction

            if not momentum_predictions:
                print("  [SKIP] No models produced valid predictions for this date.")
                continue

            # --- Scoring, Ranking, and Saving Logic (Unchanged) ---
            def momentum_score(prediction):
                if not prediction: return -999
                trend_strength = prediction['trend_strength']
                consistency = 1 - prediction['directional_consistency']
                persistence = prediction['momentum_persistence']
                return (0.4 * trend_strength + 0.3 * consistency + 0.3 * persistence)

            ranking_data = []
            for symbol, pred in momentum_predictions.items():
                score = momentum_score(pred)
                ranking_data.append({'symbol': symbol, 'momentum_score': score, **pred})
            
            ranking_df = pd.DataFrame(ranking_data).sort_values('momentum_score', ascending=False)
            ranking_df['rank'] = range(1, len(ranking_df) + 1)
            
            # Save CSV file named with the date
            csv_filename = f"momentum_ranking_{prediction_date_str}.csv"
            ranking_df.head(10).to_csv(backtest_dir / csv_filename, index=False)
            
            print(f"  Saved: {csv_filename} ({len(ranking_df.head(10))} stocks)")

        except Exception as e:
            print(f"  [ERROR] Failed for {prediction_date_str}: {e}")

    print(f"\n{'='*60}\nINTRADAY BACKTEST COMPLETED\n{'='*60}")
if __name__ == "__main__":
    generate_backtest_csvs()

Fetching data from 2024-05-24 to 2025-10-10
Loading local 15-minute data from: combined_stock_data 📂

Successfully loaded and processed data for 90 tickers.

Found 264 unique trading days in the data. Starting backtest...

Processing 1/264: 2024-09-02
Trained momentum model for ABB.NS - 243 samples
Trained momentum model for ADANIENT.NS - 245 samples
Trained momentum model for ADANIGREEN.NS - 246 samples
Trained momentum model for ADANIPORTS.NS - 247 samples
Trained momentum model for ADANIPOWER.NS - 242 samples
Trained momentum model for AMBUJACEM.NS - 246 samples
Trained momentum model for APOLLOHOSP.NS - 245 samples
Trained momentum model for ASIANPAINT.NS - 246 samples
Trained momentum model for DMART.NS - 243 samples
Trained momentum model for AXISBANK.NS - 248 samples
Trained momentum model for BAJAJ-AUTO.NS - 241 samples
Trained momentum model for BAJFINANCE.NS - 247 samples
Trained momentum model for BAJAJFINSV.NS - 248 samples
Trained momentum model for BAJAJHLDNG.NS - 234 sam